In [ ]:
from osgeo import gdal
import numpy as np

In [ ]:
def tif2npy(tif_path, output_path, image_class):
    ds = gdal.Open(tif_path)
    if ds is None:
        print("No such file !")

    image = ds.ReadAsArray(0, 0, ds.RasterXSize, ds.RasterYSize)

    # 有效范围截断
    if image_class == "S1":
        np.clip(image, -50, 1, out=image)
    elif image_class == "S2":
        np.clip(image, 0, 10000, out=image)
    
    if image_class == "S2":
        selected_bands = [1, 2, 3, 4, 7, 8, 10, 11]
    elif image_class == "S1":
        selected_bands = [0, 1]
    elif image_class == "MODIS":
        selected_bands = [0, 1, 2, 3, 5, 6]

    image = image[selected_bands, :, :]
    # S2: (8, 250, 250), S1: (2, 250, 250), MODIS: (6, 5, 5)
    np.save(output_path, image)

    ds = None

    print("Successfully convert", tif_path, "to", output_path)
    
    return True

In [ ]:
import os

def get_files_by_type(folder_path, file_type):
    file_paths = []
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.endswith(file_type):
                file_paths.append(os.path.join(root, file))
    return file_paths

def categorize_files_by_keyword(file_paths, keywords):
    categorized_files = {keyword: [] for keyword in keywords}
    
    for file_path in file_paths:
        for keyword in keywords:
            if keyword in file_path:
                categorized_files[keyword].append(file_path)
    
    return categorized_files

folder_path = r"D:\ENVI\data\NingBo\NB_roi\cropped_data"

file_type = ".tif"

keywords = ["MODIS", "S1", "S2"]

file_paths = get_files_by_type(folder_path, file_type)

categorized_files = categorize_files_by_keyword(file_paths, keywords)

# for keyword, files in categorized_files.items():
#     print(f"Files with '{keyword}' in the path:")
#     for file in files:
#         print(file)
#     print("\n")


In [ ]:
for keyword, files in categorized_files.items():
    print(keyword)
    for file_path in files:
        output_path = file_path.replace(r"D:\ENVI\data\NingBo\NB_roi\cropped_data", r"D:\Code\MODIS_S1_S2\dataset\SatelliteImages").replace("tif", "npy")
        tif2npy(file_path, output_path, image_class=keyword)

In [ ]:
image = np.load(r"D:\Code\MODIS_S1_S2\dataset\SatelliteImages\train\MODIS\MODIS_0.npy")

min_val = np.min(image)
max_val = np.max(image)
mean = np.mean(image.reshape(-1))
std = np.std(image.reshape(-1))

print(min_val, max_val, mean, std)

In [1]:
import sys
sys.path.append(r'D:\Code\MODIS_S1_S2\src')

import glob
from utils import calc_statistics
import numpy as np

MODIS_dir = r"D:\Code\MODIS_S1_S2\dataset\SatelliteImages\train\MODIS\MODIS_*.npy"
S1_dir = r"D:\Code\MODIS_S1_S2\dataset\SatelliteImages\train\S1\S1_*.npy"
S2_dir = r"D:\Code\MODIS_S1_S2\dataset\SatelliteImages\train\S2\S2_*.npy"

MODIS_image_paths = glob.glob(MODIS_dir)
S1_image_paths = glob.glob(S1_dir)
S2_image_paths = glob.glob(S2_dir)


# min_val, max_val, mean, std = calc_statistics(MODIS_image_paths)
# print("MODIS: ")
# print(f"min: {min_val}\nmax: {max_val}\nmean: {mean}\nstd: {std}")

# min_val, max_val, mean, std = calc_statistics(S1_image_paths)
# print("S1: ")
# print(f"min: {min_val}\nmax: {max_val}\nmean: {mean}\nstd: {std}")

# min_val, max_val, mean, std = calc_statistics(S2_image_paths[:1500])
# print("S2: ")
# print(f"min: {min_val}\nmax: {max_val}\nmean: {mean}\nstd: {std}")

S2: 
min: [0 0 0 0 0 0 0 0]
max: [10000 10000 10000 10000 10000 10000 10000 10000]
mean: [ 978.62840361 1200.53972316 1157.46839859 1548.37047019 2613.07640004
 2679.20647717 2024.73282798 1522.60639281]
std: [ 576.94529047  588.16202837  634.95581889  635.59533807 1116.8635003
 1132.44639616  822.60875038  753.66252027]
